<a href="https://colab.research.google.com/github/emoceanographer/usdanutrients/blob/master/Nutrition_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import urllib
import pandas as pd

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Nutrition/'
search_path = path + 'fao_search_terms.xlsx'

fao_search_df = pd.read_excel(search_path)

fao_search = fao_search_df[fao_search_df['Exclude'].isna()]

In [0]:
fao_items = fao_search['Item'].unique().tolist()

In [0]:
# Pull which data we want from it
# 'ru': reporting unit
# nutrient_id: 208 (energy)
# nutrient_id: 203 (protein) -- value, dp (datapoints), se
# any one with "group": "Amino Acids"
  # grab "value"  in the first box
  
aa_list = ['Tryptophan', 'Threonine', 'Isoleucine', 'Leucine', 'Lysine',
          'Methionine', 'Cystine', 'Phenylalanine', 'Tyrosine', 'Valine',
          'Arginine', 'Histidine', 'Alanine', 'Aspartic', 'Glutamic', 
          'Glycine', 'Serine', 'Hydroxyproline']


In [0]:
# Set-up for USDA API
my_key = 'owv7XVVwwlt6tZQvEMSkZvLIEQiElEGwioNElEaH'
url_search_start = 'https://api.nal.usda.gov/ndb/search/?format=json&q='
url_search_end = '&ds=Standard%20Reference&max=1500&lt=f&sort=r&api_key='
# sort = r gives by relevance; n changes to by name


In [0]:
  #temp = {'list': {'q': '', 'sr': '1', 'ds': 'Standard Reference', 'start': 0, 'end': 1500, 'total': 7793}}
  #print(temp.get('end'))
  #print(type(temp.get('end')))
  #url = url_search_start + query + '&offset='+ temp.get('end') + url_search_end + my_key
  #url

In [0]:
def usda_query(query):
  url = url_search_start + query + url_search_end + my_key

  data = urllib.request.urlopen(url)
  data = json.load(data)

  data = data.get('list')
  if data:
    data = data.get('item') # deletes the search summary
    if len(data)>1499: # this is the maximum the API can return
      print(['for ', query, 'items exceeded max threshold'])
  else:
    data = data
  
  return data
  

In [118]:
fao_search.head()

,Item,Search terms,Exclude
1,Apples and products,'apple',NaN
3,Aquatic Plants,"'seaweed','algae'",NaN
4,Bananas,'banana',NaN
5,Barley and products,'barley',NaN
6,Beans,'beans',NaN


In [223]:
fao_items

['Apples and products',
 'Aquatic Plants',
 'Bananas',
 'Barley and products',
 'Beans',
 'Beer',
 'Bovine Meat',
 'Butter, Ghee',
 'Cassava and products',
 'Cephalopods',
 'Cocoa Beans and products',
 'Coconut Oil',
 'Coconuts - Incl Copra',
 'Coffee and products',
 'Cottonseed',
 'Cottonseed Oil',
 'Cream',
 'Crustaceans',
 'Dates',
 'Demersal Fish',
 'Eggs',
 'Fats, Animals, Raw',
 'Freshwater Fish',
 'Grapefruit and products',
 'Grapes and products (excl wine)',
 'Groundnut Oil',
 'Groundnuts (Shelled Eq)',
 'Honey',
 'Lemons, Limes and products',
 'Maize and products',
 'Maize Germ Oil',
 'Meat, Aquatic Mammals',
 'Milk - Excluding Butter',
 'Millet and products',
 'Mutton & Goat Meat',
 'Nuts and products',
 'Oats',
 'Offals, Edible',
 'Olive Oil',
 'Olives (including preserved)',
 'Onions',
 'Oranges, Mandarines',
 'Palm kernels',
 'Palm Oil',
 'Palmkernel Oil',
 'Peas',
 'Pelagic Fish',
 'Pigmeat',
 'Pimento',
 'Pineapples and products',
 'Plantains',
 'Potatoes and products',


#USDA API

In [119]:
# Runs queries through the USDA API and pulls lists of items matching
search_results = {}
for element in fao_items:
  print(element)
  temp_results = []
  fao_item = fao_search[fao_search['Item']==element]
  search_terms = fao_item['Search terms'].to_list()
  search_terms = search_terms[0].split(',')

  for subelement in search_terms:
    query = subelement
    temp = usda_query(query)
    if temp:
      temp_results = temp_results + temp
  search_results[element] = temp_results


Apples and products
Aquatic Plants
Bananas
Barley and products
Beans
Beer
Bovine Meat
Butter, Ghee
Cassava and products
Cephalopods
Cocoa Beans and products
Coconut Oil
Coconuts - Incl Copra
Coffee and products
Cottonseed
Cottonseed Oil
['for ', "'cottonseed20%oil'", 'items exceeded max threshold']
Cream
Crustaceans
Dates
Demersal Fish
Eggs
Fats, Animals, Raw
Freshwater Fish
Grapefruit and products
Grapes and products (excl wine)
Groundnut Oil
Groundnuts (Shelled Eq)
Honey
Lemons, Limes and products
Maize and products
Maize Germ Oil
Meat, Aquatic Mammals
Milk - Excluding Butter
Millet and products
Mutton & Goat Meat
Nuts and products
Oats
Offals, Edible
Olive Oil
Olives (including preserved)
Onions
Oranges, Mandarines
Palm kernels
Palm Oil
Palmkernel Oil
Peas
Pelagic Fish
Pigmeat
Pimento
Pineapples and products
Plantains
Potatoes and products
Poultry Meat
Pulses, Other and products
Rape and Mustard Oil
['for ', '', 'items exceeded max threshold']
Rape and Mustardseed
Rice (Milled Equiv

In [120]:
# makes dataframe from the data
ctr = 0
commodities = []
all_items = search_results.keys()
for element in all_items: 
  df = pd.DataFrame.from_dict(search_results[element])
  df['query'] = element
  if ctr == 0:
    commodities = df
    ctr = 1
  else:
    commodities = commodities.append(df)

  
    


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [121]:
# remove duplicates
commodities_dups = commodities
commodities_dups.drop_duplicates(subset = 'ndbno',keep = 'first')

,ds,group,manu,name,ndbno,offset,query
0,SR,Baked Products,none,"Croissants, apple",18240,0.0,Apples and products
1,SR,Baked Products,none,"Strudel, apple",18354,1.0,Apples and products
2,SR,Baby Foods,none,"Babyfood, apples, dices, toddler",03115,2.0,Apples and products
3,SR,Baby Foods,none,"Babyfood, juice, apple",03166,3.0,Apples and products
4,SR,Baby Foods,none,"Babyfood, apple-banana juice",03167,4.0,Apples and products
...,...,...,...,...,...,...,...
1,SR,Vegetables and Vegetable Products,none,"Mountain yam, hawaii, raw",11258,1.0,Yams
2,SR,Vegetables and Vegetable Products,none,"Mountain yam, hawaii, cooked, steamed, without...",11259,2.0,Yams
3,SR,Vegetables and Vegetable Products,none,"Mountain yam, hawaii, cooked, steamed, with salt",11927,3.0,Yams
4,SR,Vegetables and Vegetable Products,none,"Yam, cooked, boiled, drained, or baked, withou...",11602,4.0,Yams


In [0]:
commodities_dups[commodities_dups['query'].isin(['Groundnuts (Shelled Eq)'])] # still has peanuts

In [0]:
savepath = '/content/drive/My Drive/Colab Notebooks/Nutrition/' + 'usda_items_112619.csv'
commodities_dups.to_csv(savepath)

# Compare new run with previously annotated data
The previously annotated data should override / keep its annotations; add new data with a flag that it needs manual annotation


In [0]:
import numpy as np

In [124]:
commodities_dups['Raw'] = np.nan
commodities_dups['Cooked'] = np.nan
commodities_dups['Exclude'] = np.nan
commodities_dups.head()

,ds,group,manu,name,ndbno,offset,query,Raw,Cooked,Exclude
0,SR,Baked Products,none,"Croissants, apple",18240,0.0,Apples and products,NaN,NaN,NaN
1,SR,Baked Products,none,"Strudel, apple",18354,1.0,Apples and products,NaN,NaN,NaN
2,SR,Baby Foods,none,"Babyfood, apples, dices, toddler",03115,2.0,Apples and products,NaN,NaN,NaN
3,SR,Baby Foods,none,"Babyfood, juice, apple",03166,3.0,Apples and products,NaN,NaN,NaN
4,SR,Baby Foods,none,"Babyfood, apple-banana juice",03167,4.0,Apples and products,NaN,NaN,NaN


In [0]:
most_recent_annotated = 'data_102919.csv'
openpath = path + most_recent_annotated #adds columns for 
  # "raw", "cooked", "exclude"
  
data = pd.read_csv(openpath)

In [0]:
# Compare loaded / annotated data with the recently generated commodities_dups
# gets identifiers of the commodities not in the annotated list
new_ndbno = list(commodities_dups.ndbno.unique()) # gets the unique identifiers of the annotated data

new_idx = []
for item_no in new_ndbno: 
  temp_df = data[data['ndbno'].isin([item_no])]
  if not len(temp_df)>0: 
    new_idx.append(item_no)
 
    


In [0]:
new_items = commodities_dups[commodities_dups['ndbno'].isin(new_idx)] # these are the new items

In [128]:
data_all = pd.concat([data,new_items]) # merges the two datasets without annotation for the new_items

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
# Minor cleaning
data_all.loc[0,'Cooked'] = 0

#'query' = name of category
#'name' = name of food
#'Raw' = EAM annotation for raw
#'Cooked' = EAM annotation for cooked / processed
#'Exclude' = EAM annotation for not category or multiple foods
#'ndbno' = the API reference number

## Add annotations to the new entries

In [0]:
savepath = '/content/drive/My Drive/Colab Notebooks/Nutrition/' + 'new_items_112619.csv'
new_items.to_csv(savepath)

In [0]:
# ANNOTATE MANUALLY
new_path = '/content/drive/My Drive/Colab Notebooks/Nutrition/' + 'new_items_112619_a.csv'
new_annotated = pd.read_csv(new_path)

In [132]:
data_all = data.append(new_annotated)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [0]:
data = data_all # assigns the merged dataset to "data"

In [0]:
savepath = '/content/drive/My Drive/Colab Notebooks/Nutrition/' + 'data_112619.csv'
data.to_csv(savepath)

In [0]:
# Opens saved dataframe
savepath = '/content/drive/My Drive/Colab Notebooks/Nutrition/' + 'data_112619.csv'
data = pd.read_csv(savepath)
data = data[['ds','group','manu','name','ndbno','offset','query','Raw','Cooked','Exclude']]

In [146]:
data[data['query'].isin(['Groundnuts (Shelled Eq)'])] # STILL HAS GROUNDNUTS

,ds,group,manu,name,ndbno,offset,query,Raw,Cooked,Exclude
7898,SR,Legumes and Legume Products,none,"Peanuts, all types, raw",16087,2,Groundnuts (Shelled Eq),1.0,0.0,NaN
7899,SR,Legumes and Legume Products,none,"Peanuts, spanish, raw",16091,3,Groundnuts (Shelled Eq),1.0,0.0,NaN
7900,SR,Legumes and Legume Products,none,"Peanuts, valencia, raw",16093,4,Groundnuts (Shelled Eq),1.0,0.0,NaN
7901,SR,Legumes and Legume Products,none,"Peanuts, virginia, raw",16095,5,Groundnuts (Shelled Eq),1.0,0.0,NaN
7902,SR,Legumes and Legume Products,none,"Peanut flour, defatted",16099,6,Groundnuts (Shelled Eq),1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
7956,SR,Snacks,none,"Snacks, popcorn, caramel-coated, with peanuts",19038,80,Groundnuts (Shelled Eq),0.0,0.0,1.0
7957,SR,Snacks,none,"Snacks, popcorn, caramel-coated, without peanuts",19039,81,Groundnuts (Shelled Eq),0.0,0.0,1.0
7958,SR,Snacks,none,"Snacks, granola bars, hard, peanut butter",19420,82,Groundnuts (Shelled Eq),0.0,0.0,1.0
7959,SR,Snacks,none,"Formulated bar, MARS SNACKFOOD US, SNICKERS MA...",25004,83,Groundnuts (Shelled Eq),0.0,0.0,1.0


In [0]:
# re-cast data into a dict for easy manipulation
data_dict = data.to_dict(orient = 'records')

In [0]:
fao_categories = list(fao_search.keys()) # gets just the keys / category names

In [0]:
fao_categories = fao_items

In [153]:
data_dict

[{'Cooked': 0.0,
  'Exclude': nan,
  'Raw': 1.0,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley, hulled',
  'ndbno': 20004,
  'offset': 0,
  'query': 'Barley and products'},
 {'Cooked': 0.0,
  'Exclude': nan,
  'Raw': 1.0,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley, pearled, raw',
  'ndbno': 20005,
  'offset': 1,
  'query': 'Barley and products'},
 {'Cooked': 1.0,
  'Exclude': nan,
  'Raw': 0.0,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley, pearled, cooked',
  'ndbno': 20006,
  'offset': 2,
  'query': 'Barley and products'},
 {'Cooked': 0.0,
  'Exclude': nan,
  'Raw': 1.0,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley flour or meal',
  'ndbno': 20130,
  'offset': 3,
  'query': 'Barley and products'},
 {'Cooked': 0.0,
  'Exclude': nan,
  'Raw': 1.0,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',


#Pull dietary information

In [0]:
# pull the last run so we can only run those that are missing
foods_output_last = pd.read_csv(path + 'foods_output_1126_3missing.csv')

In [0]:
new_only = 'no' # yes doesn't currently work

In [0]:
def data_pull(food_dict):
  nutr_store = {}
  for element in food_dict:
    if element['nutrient_id'] == 208: # energy in kCal
      nutr_store['energy'] = element['value']
      #unit_check(element)
    if element['nutrient_id'] == 203: # protein in g
      nutr_store['protein'] = element['value']
      #unit_check(element)
    if element['nutrient_id'] == 303: # iron in mg
      nutr_store['iron'] = element['value']
    if element['nutrient_id'] == 309: # zinc
      nutr_store['zinc'] = element['value']
    if element['nutrient_id'] == 415: # B-6
      nutr_store['B6'] = element['value']
    if element['nutrient_id'] == 418: # B-12
      nutr_store['B12'] = element['value']
      #protein_se = element['sourcecode']['se']
    for aa in aa_list: # cycle through each amino acid
      if aa in element['name']:
        nutr_store[aa] = element['value']
        #unit_check(element)
  return nutr_store

In [0]:
def unit_check(element):
  """Checks if the measure lists g"""
  if not element['unit'] in 'g':
    print('unit error')
    print(element)
    

In [0]:
# Code for pulling a report, not a search
url_report_start = 'https://api.nal.usda.gov/ndb/V2/reports?ndbno='

url_report_end = '&type=f&format=json&api_key='
# type = f gives full report

In [0]:
my_key2 = '6pPcG0tjgEeIdMpD3TnSyoq42NKrmohXyY0MMT0Q'

In [187]:
# pulling data from USDA -- new run only
if new_only == 'yes':
  food_dict = {}
  for item in data_dict:
    temp_dict = {}
    if item['Exclude'] != 1: # checks if it was included
      if item['Raw'] == 1 or item['Raw'] == '1':
        status = 'raw'
      if item['Cooked'] == '1' or item['Cooked'] == 1:
        status = 'cooked'
      item_number = str(item['ndbno'])
      if len(item_number) < 5: # indicates a 0 was lost from excel
        item_number = '0'+item_number

      if not int(item_number) in list(foods_output_last['item_no'].unique()):
      # selects only those NOT in our previous database pull     
        name = item['name']

        url = url_report_start + item_number + url_report_end + my_key
        nutr_data = urllib.request.urlopen(url)
        nutr_data = json.load(nutr_data)

        temp = nutr_data['foods'][0]['food']['nutrients']
        nutrients = data_pull(temp)

        food_group = item['query']

        if not food_group in food_dict.keys():
          food_dict[food_group] = []
          print(food_group)

        #if item['query'] == (['Groundnuts (Shelled Eq)']):
          #break

        temp_dict[name] = nutrients
        temp_dict[name]['item_no'] = item_number

        temp_dict[name]['units'] = nutr_data['foods'][0]['food']['desc']['ru']
        temp_dict[name]['descr'] = nutr_data['foods'][0]['food']['desc']['sd']
        temp_dict[name]['status'] = status
        food_dict[food_group].append(temp_dict)

Pimento
Pineapples and products
Tomatoes and products


In [0]:
foods_temp = foods_output_last
foods_temp = foods_temp.set_index('food')
foods_temp = foods_temp.drop_duplicates()

In [211]:
if new_only == 'yes':
  foods_dict_last = foods_temp.to_dict('index')

foods_dict_last

ValueError: ignored

In [217]:
# pulling items from data_dict and running through USDA; ALL run
if not new_only == 'yes':
  food_dict = {}
  for item in data_dict:
    temp_dict = {}
    if item['Exclude'] != 1: # checks if it was included
      if item['Raw'] == 1 or item['Raw'] == '1':
        status = 'raw'
      if item['Cooked'] == '1' or item['Cooked'] == 1:
        status = 'cooked'
      item_number = str(item['ndbno'])
      if len(item_number) < 5: # indicates a 0 was lost from excel
        item_number = '0'+item_number

      name = item['name']
      #print(name)
      url = url_report_start + item_number + url_report_end + my_key
      nutr_data = urllib.request.urlopen(url)
      nutr_data = json.load(nutr_data)

      temp = nutr_data['foods'][0]['food']['nutrients']
      nutrients = data_pull(temp)

      food_group = item['query']

      if not food_group in food_dict.keys():
        food_dict[food_group] = []
        print(food_group)

      #if item['query'] == (['Groundnuts (Shelled Eq)']):
        #break

      temp_dict[name] = nutrients
      temp_dict[name]['item_no'] = item_number

      temp_dict[name]['units'] = nutr_data['foods'][0]['food']['desc']['ru']
      temp_dict[name]['descr'] = nutr_data['foods'][0]['food']['desc']['sd']
      temp_dict[name]['status'] = status
      food_dict[food_group].append(temp_dict)
 

Barley and products
Beans
Aquatic plants
Bovine meat
Butter ghee
Cassava and products
Cephalopods
Coconut Oil
Coconuts - Incl Copra
Cottonseed
Cottonseed Oil
Cream
Crustaceans
Dates
Demersal Fish
Eggs
Fish Body Oil
Freshwater Fish
Honey
Maize and products
Meat Aquatic Mammals
Milk - Excluding Butter
Millet and products
Molluscs other
Mutton & Goat Meat
Nuts and products
Oats
Olive Oil
Palm kernels
Peas
Pigmeat
Plantains
Potatoes and products
Poultry Meat
Pulses
Rape and Mustard Oil
Rice (Milled Equivalent)
Rye and products
Sesame seed
Sorghum and products
Soyabean Oil
Soyabeans
Starchy Roots
Sunflower seed
Sunflowerseed Oil
Sweet potatoes
treenuts
wheat
yams
Groundnuts (Shelled Eq)
Apples and products
Bananas
Beer
Cocoa Beans and products
Coffee and products
Grapefruit and products
Grapes and products (excl wine)
Lemons, Limes and products
Onions
Oranges, Mandarines
Pimento
Pineapples and products
Tomatoes and products


In [0]:
category_averages = []
header = aa_list + ['energy', 'B12', 'B6', 'protein', 'iron', 'item_no', 'status']

ctr = 0
category_averages = []

for element in food_dict: 
  df = []
  for item in food_dict[element]: 
    name = list(item.keys())[0]   
    attrs = [name]
    #print(item)
    for attr in header:
      attr_value = item[name].get(attr)
      attrs.append(attr_value)
    df.append(attrs)
  test = pd.DataFrame.from_records(df,columns = ['food'] + header)
  
  raw_avg = []
  raw_std = []
  cooked_avg = []
  cooked_std = []
  for attr in header:
    if not attr in 'item_no' and not attr in 'status': # don't run non-numeric values
      raw_avg.append(test.loc[test['status'] == 'raw'][attr].mean(skipna=True))
      raw_std.append(test.loc[test['status'] == 'raw'][attr].std(skipna=True))
      cooked_avg.append(test.loc[test['status'] == 'cooked'][attr].mean(skipna=True))
      cooked_std.append(test.loc[test['status'] == 'cooked'][attr].std(skipna=True))
  
  #print(raw_avg)
  if ctr == 0:
    total_df = test
    ctr = 1
  else:
    total_df = total_df.append(test)


  #print(list(map(str,raw_avg)))
  category_averages.append([element, 'raw', 'avg'] + list(map(str,raw_avg)))
  category_averages.append([element, 'raw', 'std'] + list(map(str,raw_std)))
  category_averages.append([element, 'cooked', 'avg'] + list(map(str,cooked_avg)))
  category_averages.append([element, 'cooked', 'std'] + list(map(str,cooked_std)))


    


In [0]:
nutrient_summary = pd.DataFrame.from_records(category_averages,columns = ['food', 'status', 'avg/std'] + header[0:len(header)-2])


In [0]:
savepath = '/content/drive/My Drive/Colab Notebooks/Nutrition/' + 'nutrient_output_1126.csv'
nutrient_summary.to_csv(savepath)

In [0]:
savepath = path + 'foods_output_1126.csv'
total_df.to_csv(savepath)

# Extraneous

In [0]:
test = pd.DataFrame.from_records(df,columns = ['food'] + header)
test.loc[test['status'] == 'raw']['Valine'].mean()
#test.loc[test['status'] == 'cooked'].mean(axis=0)

0.44539999999999996

In [0]:
for element in food_dict: # cycles through each food group
  print(food_dict[element])
  element_df = pd.DataFrame.from_dict(food_dict[element][0],orient='index')
  break
  
    if ctr == 0:
    category_averages = df
    ctr = 1
  else:
    category_averages = category_averages.append(df)

In [0]:
element_df

,energy,protein,iron,zinc,B6,B12,Tryptophan,Threonine,Isoleucine,Leucine,Lysine,Methionine,Cystine,Phenylalanine,Tyrosine,Valine,Arginine,Histidine,Alanine,Aspartic,Glutamic,Glycine,Serine,item_no,units,descr,status
"Barley, hulled",354.0,12.48,3.6,2.77,0.318,0.0,0.208,0.424,0.456,0.848,0.465,0.24,0.276,0.7,0.358,0.612,0.625,0.281,0.486,0.779,3.261,0.452,0.527,20004,g,"BARLEY,HULLED",raw


In [0]:
food_dict = {}
for item in barley: 
  item_number = str(item['ndbno'])
  if len(item_number) < 5: # indicates a 0 was lost from excel
    item_number = '0'+item_number
  
  name = item['name']
  print(name)
  url = url_report_start + item_number + url_report_end + my_key
  nutr_data = urllib.request.urlopen(url)
  nutr_data = json.load(nutr_data)
  
  temp = nutr_data['foods'][0]['food']['nutrients']
  nutrients = data_pull(temp)
  
  food_dict[name] = nutrients
  food_dict[name]['item_no'] = item_number
  
  food_dict[name]['units'] = nutr_data['foods'][0]['food']['desc']['ru']
  food_dict[name]['descr'] = nutr_data['foods'][0]['food']['desc']['sd']


Barley, hulled
Barley, pearled, raw
Barley flour or meal
Barley malt flour
Babyfood, cereal, barley, dry fortified


In [0]:
df = pd.DataFrame.from_dict(food_dict, orient = 'index', dtype=None, columns=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986
Barley and products,"{'Barley, hulled': {'energy': 354.0, 'protein'...","{'Barley, pearled, raw': {'energy': 352.0, 'pr...","{'Barley, pearled, cooked': {'energy': 123.0, ...","{'Barley flour or meal': {'energy': 345.0, 'pr...","{'Barley malt flour': {'energy': 361.0, 'prote...","{'Babyfood, cereal, barley, dry fortified': {'...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Beans,"{'Beans, snap, green, raw': {'energy': 31.0, '...","{'Beans, snap, green, microwaved': {'energy': ...","{'Yardlong bean, raw': {'energy': 47.0, 'prote...","{'Winged bean leaves, raw': {'energy': 74.0, '...","{'Winged bean tuber, raw': {'energy': 148.0, '...","{'Beans, snap, yellow, raw': {'energy': 31.0, ...","{'Beans, baked, home prepared': {'energy': 155...","{'Chili with beans, canned': {'energy': 103.0,...","{'Beans, shellie, canned, solids and liquids':...","{'Hyacinth-beans, immature seeds, raw': {'ener...","{'Beans, fava, in pod, raw': {'energy': 88.0, ...","{'Beans, adzuki, mature seeds, raw': {'energy'...","{'Beans, baked, canned, plain or vegetarian': ...","{'Beans, black, mature seeds, raw': {'energy':...","{'Beans, french, mature seeds, raw': {'energy'...","{'Beans, navy, mature seeds, raw': {'energy': ...","{'Beans, navy, mature seeds, canned': {'energy...","{'Beans, pink, mature seeds, raw': {'energy': ...","{'Beans, pinto, mature seeds, raw (Includes fo...","{'Beans, yellow, mature seeds, raw': {'energy'...","{'Beans, white, mature seeds, raw': {'energy':...","{'Beans, white, mature seeds, canned': {'energ...","{'Beans, pinto, canned, drained solids': {'ene...","{'Restaurant, Latino, black bean soup': {'ener...","{'Beans, kidney, mature seeds, sprouted, raw':...","{'Beans, navy, mature seeds, sprouted, raw': {...","{'Beans, pinto, immature seeds, frozen, unprep...","{'Beans, snap, green, frozen, all styles, unpr...","{'Beans, snap, green, frozen, all styles, micr...","{'Yardlong bean, cooked, boiled, drained, with...","{'Beans, pinto, mature seeds, sprouted, raw': ...","{'Beans, snap, yellow, frozen, all styles, unp...","{'Yardlong bean, cooked, boiled, drained, with...","{'Beans, adzuki, mature seeds, canned, sweeten...","{'Beans, baked, canned, with pork and sweet sa...","{'Beans, baked, canned, with pork and tomato s...","{'Beans, black turtle, mature seeds, raw': {'e...","{'Beans, black turtle, mature seeds, canned': ...","{'Beans, cranberry (roman), mature seeds, raw'...","{'Beans, cranberry (roman), mature seeds, cann...",...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aquatic plants,"{'Seaweed, agar, raw': {'energy': 26.0, 'prote...","{'Seaweed, irishmoss, raw': {'energy': 49.0, '...","{'Seaweed, kelp, raw': {'energy': 43.0, 'prote...","{'Seaweed, laver, raw': {'energy': 35.0, 'prot...","{'Seaweed, agar, dried': {'energy': 306.0, 'pr...","{'Seaweed, spirulina, raw': {'energy': 26.0, '...","{'Seaweed, spirulina, dried': {'energy': 290.0...","{'Seaweed, wakame, raw': {'energy': 45.0, 'pro...","{'Seaweed, Canadian Cultivated EMI-TSUNOMATA, ...","{'Seaweed, Canadian Cultivated EMI-TSUNOMATA, ...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,

In [0]:
savepath = '/content/drive/My Drive/Colab Notebooks/Nutrition/' + 'nutrient_test.csv'
df.to_csv(savepath)

In [0]:
nutr_store = {}
for element in temp:
  if element['nutrient_id'] == 208: # energy in kCal
    nutr_store['energy'] = element['value']
  if element['nutrient_id'] == 203: # protein in g
    nutr_store['protein'] = element['value']
    #protein_se = element['sourcecode']['se']
  for aa in aa_list: # cycle through each amino acid
    if aa in element['name']:
      nutr_store[aa] = element['value']

In [0]:
# barley raw sample
barley = data.iloc[[0,1,3,4,5]]
barley = barley.to_dict(orient = 'records')

In [0]:
barley

[{'Cooked': 0,
  'Exclude': nan,
  'Raw': 1,
  'Unnamed: 0': 0,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley, hulled',
  'ndbno': 20004,
  'offset': 0,
  'query': 'Barley and products'},
 {'Cooked': '0',
  'Exclude': nan,
  'Raw': 1,
  'Unnamed: 0': 1,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley, pearled, raw',
  'ndbno': 20005,
  'offset': 1,
  'query': 'Barley and products'},
 {'Cooked': '0',
  'Exclude': nan,
  'Raw': 1,
  'Unnamed: 0': 3,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley flour or meal',
  'ndbno': 20130,
  'offset': 3,
  'query': 'Barley and products'},
 {'Cooked': '0',
  'Exclude': nan,
  'Raw': 1,
  'Unnamed: 0': 4,
  'ds': 'SR',
  'group': 'Cereal Grains and Pasta',
  'manu': 'none',
  'name': 'Barley malt flour',
  'ndbno': 20131,
  'offset': 4,
  'query': 'Barley and products'},
 {'Cooked': '0',
  'Exclude': nan,
  'Raw': 1,
  'Unnamed: 

In [0]:
# I think this is all extraneous


url = url_report_start + query + url_report_end + my_key
nutr_data = urllib.request.urlopen(url)
nutr_data = json.load(nutr_data)

temp = nutr_data['foods'][0]['food']['nutrients']

NameError: ignored